这是一个多分类（multilabel, multiclass classification）的问题，同一条心电图曲线，可能出现多种心率失常的状态。
由于心电图的特性，使用类似时间序列的模型最佳

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
# constant
root_dir = r'C:/D/nsmc/TianChi/HeFei'
train_dir = os.path.join(root_dir, 'train')
testa_dir = os.path.join(root_dir, 'testA')
train_label_file = os.path.join(root_dir, 'hf_round1_label.txt')
arrythmia_file = os.path.join(root_dir, 'hf_round1_arrythmia.txt')

#### 加载train_data

In [ ]:
#I II V1 V2 V3 V4 V5 V6 
train_files = os.listdir(train_dir)
train_files.sort()
length = len(train_files)
train_data = np.zeros((length, 5000, 8))
for i, train_file in enumerate(train_files):
    data = np.loadtxt(one_train_file, skiprows=1)[np.newaxis, :]
    train_data[i, :] = data
    
# III=II-I
# aVR=-(I+II)/2
# aVL=I-II/2
# aVF=II-I/2
train_data_extend = np.zeros((length, 5000, 4))
train_data_entend[:, 0] = train_data[:, 1] - train_data[:, 0]
train_data_entend[:, 1] = (train_data[:, 0] + train_data[:, 1]) / 2
train_data_entend[:, 2] = (train_data[:, 0] - train_data[:, 1]) / 2
train_data_entend[:, 3] = (train_data[:, 1] - train_data[:, 0]) / 2
train_data = np.concentrate((train_data, train_data_extend), axis=2)

#### 将心率失常的种类进行编码

In [ ]:
with open(arrythmia_file, 'r') as fp:
    arrythmia = [i.strip() for i in fp.readlines()]
arrythmia_encoding = {}
arrythmia_decoding = {}
for i, key in enumerate(arrythmia):
    arrythmia_encoding[key] = i
    arrythmia_decoding[i] = key

#### 将标签进行one-hot编码

In [ ]:
with open(train_label_file, 'r', encoding='utf-8') as fp:
    train_labels = [i.split('\t') for i in fp.readlines()]

In [ ]:
print(len(train_labels))
print(train_labels[0:5])

In [ ]:
def to_one_hot(labels, dimension=55):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        for j in label[3:]:
            index = arrythmia_encoding.get(j)
            if index is not None:
                results[i, index] = 1.
    return results

In [ ]:
one_hot_train_labels = to_one_hot(train_labels)

In [ ]:
print(len(one_hot_train_labels))
print(one_hot_train_labels[0:5])

In [ ]:
print('种类\t数量')
for i, count in enumerate(np.sum(one_hot_train_labels, axis=0).astype(np.int)):
    print(arrythmia_decoding[i], count)